# Importing Dependencies

In [1]:
import pandas as pd
import numpy as np
import os
import seaborn as sns

In [2]:
#! git clone https://github.com/WongKinYiu/yolov7.git

In [3]:
%cd yolov7

/home/robin/Desktop/yolov7/yolo7_kpts_xtract/yolov7


In [5]:
import torch
from torchvision import transforms

from utils.datasets import letterbox
from utils.general import non_max_suppression_kpt
from utils.plots import output_to_keypoint, plot_skeleton_kpts

import matplotlib.pyplot as plt
import cv2
import numpy as np

In [6]:
def load_model():
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model = torch.load('/home/robin/Desktop/yolov7/yolo7_kpts_xtract/yolov7/yolov7-w6-pose.pt', map_location=device)['model']
    # Put in inference mode
    model.float().eval()

    if torch.cuda.is_available():
        # half() turns predictions into float16 tensors
        # which significantly lowers inference time
        model.half().to(device)
    return model

model = load_model()

In [7]:
def run_inference(url):
    image = cv2.imread(url) # shape: (480, 640, 3)
    # Resize and pad image
    image = letterbox(image, 960, stride=64, auto=True)[0] # shape: (768, 960, 3)
    # Apply transforms
    image = transforms.ToTensor()(image) # torch.Size([3, 768, 960])
    # Turn image into batch
    image = image.unsqueeze(0) # torch.Size([1, 3, 768, 960])
    output, _ = model(image) # torch.Size([1, 45900, 57])
    return output, image

In [9]:
def visualize_output(output, image):
    output = non_max_suppression_kpt(output, 
                                     0.25, # Confidence Threshold
                                     0.65, # IoU Threshold
                                     nc=model.yaml['nc'], # Number of Classes
                                     nkpt=model.yaml['nkpt'], # Number of Keypoints
                                     kpt_label=True)
    with torch.no_grad():
        output = output_to_keypoint(output)
    nimg = image[0].permute(1, 2, 0) * 255
    nimg = nimg.cpu().numpy().astype(np.uint8)
    nimg = cv2.cvtColor(nimg, cv2.COLOR_RGB2BGR)
    
    
    
    for idx in range(output.shape[0]):
        plot_skeleton_kpts(nimg, output[idx, 7:].T, 3)
        kpts = (output[idx,7:].T)

        print(kpts)
#         landmarks = output[idx,7:].T
#         print("landmarks",landmarks)
#         print(len(landmarks))
#     plt.figure(figsize=(12, 12))
#     plt.axis('off')
#     plt.imshow(nimg)
#     plt.show()
    
    cv2.imshow("res",nimg)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [10]:
output, image = run_inference('/home/robin/Desktop/assignment/images/images/Image_4.jpg') 
visualize_output(output, image)

/home/robin/anaconda3/envs/text_extraction/lib/python3.9/site-packages/torch/functional.py:482: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3070.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


[     429.83      348.76     0.99535      421.84       283.7     0.99874      381.69      381.23     0.97683      518.47       216.3     0.91029      398.81      429.56     0.17844      729.24      231.99     0.55445      566.29      532.53     0.75505         853      198.38     0.16084      426.26      553.01
     0.33805      743.46      173.06     0.25613      317.22       376.1     0.47167      871.24      287.07    0.093104      835.27      449.38      0.1367      848.82      300.15    0.076528      816.92      490.51     0.10875      792.76      437.29    0.048305       747.1       531.6    0.060636]
